# 데이터 수집(크롤링)

## 다나와 페이지 접속

In [1]:
from selenium import webdriver
driver = webdriver.Chrome()
url = "http://search.danawa.com/dsearch.php?query=무선청소기&tab=main"
driver.get(url)

In [2]:
# html - 웹 사이트의 구성 중 하나
from bs4 import BeautifulSoup

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
#soup

In [3]:
prod_items = soup.select('li.prod_item')
len(prod_items)

42

In [4]:
title = prod_items[0].select('p.prod_name > a')[0].text.strip()
print(title)

LG전자 오브제컬렉션 코드제로 A9S AX958


In [5]:
spec_list = prod_items[0].select('div.spec_list')[0].text.replace('\t', '').replace('\n','')
print(spec_list)

핸디스틱청소기 / 무선 / 흡입+물걸레(동시) / 흡입력: 280W / [구성] 스테이션: 올인원타워4.0 / 먼지비움 / 충전 / UVC LED / 액서서리수납 / 스탠드거치 / 먼지비움시간: 30초 / 브러쉬: 와이드바닥 / 물걸레: 스팀, 고온, 일반 / 침구 / 솔형 / 틈새 / 먼지봉투: 2.5L / [배터리] 사용시간: 30분(최대) / 분리형(2개) / 리튬이온 / [청소] LED라이트 / 싸이클론흡입 / 오토스탑앤고 / 자동물공급 / 스마트인버터모터 / 워셔블헤파필터 / 크기(가로x세로x깊이): 300x1100x245mm / 출시가: 1,765,000원


In [6]:
price = prod_items[0].select('li:nth-child(1) > p.price_sect > a > strong')[0].text.strip().replace(",","")
print(price) # 71500

1089950


In [7]:
prod_items[0]

<li class="prod_item" id="productItem41827157">
<input id="productItem_categoryInfo_41827157" type="hidden" value="생활가전_청소기"/>
<input id="min_price_41827157" type="hidden" value="1089950"/>
<div class="prod_main_info">
<div class="thumb_image">
<a class="thumb_link click_log_product_standard_img_" href="https://prod.danawa.com/info/?pcode=41827157&amp;keyword=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&amp;cate=102207" id="thumbLink_41827157" onmousedown="_trkEventLog('통합검색_상품블로그_유입');" target="_blank">
<img alt="코드제로 A9S AX958" fetchpriority="high" onerror="this.onerror=null; this.src='//img.danawa.com/new/noData/img/noImg_160.gif';" src="//img.danawa.com/prod_img/500000/157/827/img/41827157_1.jpg?shrink=130:130&amp;_v=20240509152909"/>
</a>
<a class="btn_view_zoom" data-assemblygallerycategory="N" href="#" id="imageZoom_41827157" onclick="return false;">
<span class="ico i_zoom_vw"></span>이미지보기
										</a>
<input id="hidden_cate_sub_c1" type="hidden" value="72"/>
<input id="hidd

In [8]:
def get_prod_items(prod_items):
    prod_data = []
    for prod_item in prod_items:
        if 'product-pot' in prod_item['class']:
            continue
        try:
            # 1. 상품명 가져오기
            title = prod_item.select('p.prod_name > a')[0].text.strip()
            # 2. 스펙 목록 가져오기
            spec_list = prod_item.select('div.spec_list')[0].text.replace('\t', '').replace('\n','')
            # 3. 가격정보 가져오기
            price = prod_item.select('li:nth-child(1) > p.price_sect > a > strong')[0].text.strip().replace(",", "")
            prod_data.append([title, spec_list, price])
        except:
            pass
    return prod_data

In [9]:
prod_items = soup.select('div.main_prodlist > ul.product_list > li.prod_item')
print(prod_items)
prod_data = get_prod_items(prod_items)
print(len(prod_data))
#print(prod_data)

[<li class="prod_item" id="productItem41827157">
<input id="productItem_categoryInfo_41827157" type="hidden" value="생활가전_청소기"/>
<input id="min_price_41827157" type="hidden" value="1089950"/>
<div class="prod_main_info">
<div class="thumb_image">
<a class="thumb_link click_log_product_standard_img_" href="https://prod.danawa.com/info/?pcode=41827157&amp;keyword=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&amp;cate=102207" id="thumbLink_41827157" onmousedown="_trkEventLog('통합검색_상품블로그_유입');" target="_blank">
<img alt="코드제로 A9S AX958" fetchpriority="high" onerror="this.onerror=null; this.src='//img.danawa.com/new/noData/img/noImg_160.gif';" src="//img.danawa.com/prod_img/500000/157/827/img/41827157_1.jpg?shrink=130:130&amp;_v=20240509152909"/>
</a>
<a class="btn_view_zoom" data-assemblygallerycategory="N" href="#" id="imageZoom_41827157" onclick="return false;">
<span class="ico i_zoom_vw"></span>이미지보기
										</a>
<input id="hidden_cate_sub_c1" type="hidden" value="72"/>
<input id="hid

In [10]:
def get_search_page_url(keyword, page):
    return 'http://search.danawa.com/dsearch.php?query={}&volumeType=allvs&page={}&limit=30&sort=saveDESC&list=list&boost=true&addDelivery=N&tab=goods&tab=goods'.format(keyword, page)

keyword = '무선청소기'
page = 1
url = get_search_page_url(keyword, page)
print(url)

http://search.danawa.com/dsearch.php?query=무선청소기&volumeType=allvs&page=1&limit=30&sort=saveDESC&list=list&boost=true&addDelivery=N&tab=goods&tab=goods


In [11]:
import time

keyword = '무선청소기'
total_page = 10
prod_data_total = []
driver = webdriver.Chrome()

for page in range(1, total_page + 1):
    # 검색 페이지로 이동
    url = get_search_page_url(keyword, page)
    driver.get(url)

    # 페이지가 로딩 될때까지 대기
    time.sleep(5)

    # 현재 페이지의 HTML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 상품 정보 추출
    prod_items = soup.select('div.main_prodlist > ul.product_list > li.prod_item')
    prod_item_list = get_prod_items(prod_items)

    # 추출 데이터 저장
    prod_data_total = prod_data_total + prod_item_list

In [13]:
import pandas as pd
data = pd.DataFrame(prod_data_total)
data.columns = ['상품명', '스펙 목록', '가격']
data.to_excel('./1_danawa_crawling_result.xlsx', index = False)

In [14]:
prod_data_total

[['LG전자 오브제컬렉션 코드제로 A9S AX958',
  '핸디스틱청소기 / 무선 / 흡입+물걸레(동시) / 흡입력: 280W / [구성] 스테이션: 올인원타워4.0 / 먼지비움 / 충전 / UVC LED / 액서서리수납 / 스탠드거치 / 먼지비움시간: 30초 / 브러쉬: 와이드바닥 / 물걸레: 스팀, 고온, 일반 / 침구 / 솔형 / 틈새 / 먼지봉투: 2.5L / [배터리] 사용시간: 30분(최대) / 분리형(2개) / 리튬이온 / [청소] LED라이트 / 싸이클론흡입 / 오토스탑앤고 / 자동물공급 / 스마트인버터모터 / 워셔블헤파필터 / 크기(가로x세로x깊이): 300x1100x245mm / 출시가: 1,765,000원',
  '1089950'],
 ['샤오미 미홀 M22',
  '핸디스틱청소기 / 무선 / 흡입형 / [구성] 벽걸이거치 / 브러쉬: 바닥 / 물걸레: 별매 / 침구 / 솔형 / 틈새 / [배터리] 사용시간: 50분(최대) / 분리형(1개) / [청소] LED라이트 / BLDC모터 / 워셔블헤파필터 / 크기(가로x세로x깊이): 240x1040x204mm',
  '149000'],
 ['삼성전자 비스포크 제트 VS20B956AX',
  '핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 220W / [구성] 스테이션: 청정스테이션 / 먼지비움 / 충전 / UVC LED / 스탠드거치 / 먼지비움시간: 14초 / 브러쉬: 바닥 / 물걸레: 별매 / 솔형 / 틈새 / 연장관 / 먼지봉투: 1.2L / [배터리] 사용시간: 1시간(최대) / 분리형(1개) / 리튬이온 / [청소] LED라이트 / 싸이클론흡입 / 디지털인버터모터 / 워셔블헤파필터 / 크기(가로x세로x깊이): 250x930x202mm',
  '433240'],
 ['LG전자 오브제컬렉션 코드제로 A9S AX9674',
  '핸디스틱청소기 / 무선 / 흡입+물걸레(동시) / 흡입력: 250W / [구성] 스테이션: 올인원타워 / 먼지비움 / 충전 / UVC LED / 액서서리수납 / 스

In [15]:
data

,상품명,스펙 목록,가격
0,LG전자 오브제컬렉션 코드제로 A9S AX958,핸디스틱청소기 / 무선 / 흡입+물걸레(동시) / 흡입력: 280W / [구성] 스...,1089950
1,샤오미 미홀 M22,핸디스틱청소기 / 무선 / 흡입형 / [구성] 벽걸이거치 / 브러쉬: 바닥 / 물걸...,149000
2,삼성전자 비스포크 제트 VS20B956AX,핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 220W / [구성] 스테이션: 청정...,433240
3,LG전자 오브제컬렉션 코드제로 A9S AX9674,핸디스틱청소기 / 무선 / 흡입+물걸레(동시) / 흡입력: 250W / [구성] 스...,787860
4,삼성전자 비스포크 제트 AI VS25C970DW,핸디스틱청소기 / 무선 / 흡입+물걸레(교체) / 흡입력: 250W / [구성] 스...,716800
...,...,...,...
395,PRETTYCARE W400,핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 20000Pa / [구성] 브러쉬: ...,91200
396,투랩 TL19,욕실청소기 / 무선 / 전동회전 / 벽걸이거치 / 솔형 / 연장관 / 사용시간: 1...,42900
397,다이슨 V15 디텍트 앱솔루트 엑스트라,핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 240AW / [구성] 스탠드거치 /...,1065060
398,DIBEA D008 프로,핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 17000Pa / [구성] 벽걸이거치...,185900
